# Common Libraries

In [1]:
import gzip
import re
import xml.etree.ElementTree as ET
import pandas as pd

# Params

In [2]:
brodman_nifti_path = "/usr/local/afni/abin/BN_Atlas_246_1mm.nii.gz"
save_xml_data_path = "/home/seojin/Seojin_commonTool/Module/Brain_ROI/metadata.xml"

is_save_xml = True
save_info_path = "/home/seojin/Seojin_commonTool/Module/Brain_ROI/BN_Atlas_246_1mm_info.csv"

# Load data

In [3]:
with gzip.open(brodman_nifti_path, 'rt', errors='ignore') as f:
    content = f.read()

# Filtering meta data

In [4]:
metadata = re.findall(r'<AFNI_attributes.*?</AFNI_attributes>', content, re.DOTALL)[0]

if is_save_xml:
    with open(save_xml_data_path, 'w') as xml_file:
        xml_file.write(metadata)

# Read meta data

In [54]:
root = ET.fromstring(metadata)
value_label_dtable = None
for atr in root.findall(".//AFNI_atr"):
    if atr.get('atr_name') == "ATLAS_LABEL_TABLE":
        value_label_dtable = atr.text

In [66]:
pattern_corrected = r'<ATLAS_POINT.*?/>'
matches_corrected = re.findall(pattern_corrected, value_label_dtable, re.DOTALL)
matches_corrected = [e.replace("\n", "") for e in matches_corrected]

In [67]:
pattern_struct_val = r'STRUCT="([^"]+)" VAL="([^"]+)"'

In [68]:
for i, row in enumerate(matches_corrected):
    try:
        pattern_matched = re.findall(pattern_struct_val, row)[0]
    except:
        print(i, row)

39 <ATLAS_POINT data_type="atlas_point" STRUCT="A22r_left" V" "AL="79" OKEY="79" GYoAR="0" COG="54.9048 3.32622 -10.373"/>
47 <ATLAS_POINT data_type="atlas_point" STRUCT="A20" "il_left" VAL="95" OKEY="95" GYoAR="0" COG="55.8173 15.5977 -27.531"/>
93 <ATLAS_POINT data_type="atlas_point" STRUCT="cLinG_le" "ft" VAL="189" OKEY="189" GYoAR="0" COG="10.6905 82.2577 -10.7728"/>
139 <ATLAS_POINT data_type="atlas_point" STRUCT="A44op_right" "" VAL="38" OKEY="38" GYoAR="0" COG="-41.9451 -21.8587 3.4609"/>
200 <ATLAS_POINT data_type="atlas_point" STRUCT="A1/2/3tr" "u_right" VAL="162" OKEY="162" GYoAR="0" COG="-20.0271 32.6093 69.4004"/>


In [73]:
matches_corrected[39] = '<ATLAS_POINT data_type="atlas_point" STRUCT="A22r_left" VAL="79" OKEY="79" GYoAR="0" COG="54.9048 3.32622 -10.373"/>'
matches_corrected[47] = '<ATLAS_POINT data_type="atlas_point" STRUCT="A20il_left" VAL="95" OKEY="95" GYoAR="0" COG="55.8173 15.5977 -27.531"/>'
matches_corrected[93] = '<ATLAS_POINT data_type="atlas_point" STRUCT="cLinG_left" VAL="189" OKEY="189" GYoAR="0" COG="10.6905 82.2577 -10.7728"/>'
matches_corrected[139] = '<ATLAS_POINT data_type="atlas_point" STRUCT="A44op_right" VAL="38" OKEY="38" GYoAR="0" COG="-41.9451 -21.8587 3.4609"/>'
matches_corrected[200] = '<ATLAS_POINT data_type="atlas_point" STRUCT="A1/2/3tru_right" VAL="162" OKEY="162" GYoAR="0" COG="-20.0271 32.6093 69.4004"/>'
matches_corrected[-20] = '<ATLAS_POINT data_type="atlas_point" STRUCT="msOccG_right" VAL="208" OKEY="208" GYoAR="0" COG="-16.3099 85.1307 34.2996"/>'
matches_corrected[-12] = '<ATLAS_POINT data_type="atlas_point" STRUCT="NAC_right" VAL="224" OKEY="224" GYoAR="0" COG="-14.5199 -7.71109 -8.82537"/>'


In [75]:
meta_data_mapping = [re.findall(pattern_struct_val, row)[0] for row in matches_corrected]

In [79]:
meta_data_info = {}
for description, value in meta_data_mapping:
    meta_data_info[int(value)] = description

# Save data

In [88]:
save_info = pd.DataFrame({ "label" : list(meta_data_info.keys()), "description" : list(meta_data_info.values()) })
save_info.to_csv(save_info_path, index=False)

In [89]:
save_info_path

'/home/seojin/Seojin_commonTool/Module/Brain_ROI/BN_Atlas_246_1mm_info.csv'